In [1]:
import LinearAlgebra as la
using BenchmarkTools: @btime
include("hoSystem.jl");

In [2]:
import FileIO
#import Plots as plt
import ColorVectorSpace
#plt.pyplot()
include("util.jl");

## Calculating integrals and basis elements

In [3]:
l_0 = 40  # number of basis functions
ω = 0.25  # strength of harmonic oscillator potential
a = 0.25  # shielding term in coloumb interaction
grid = [x for x in range(-10, stop = 10, length = 2001)]
n = 2     # number of particles

@time system = System(n, HOBasis(l_0, ω, a), grid);

  6.399279 seconds (5.24 M allocations: 1.634 GiB, 7.38% gc time, 29.22% compilation time)


In [ ]:
1.931305 seconds (5.08 M allocations: 274.987 MiB, 5.06% gc time, 94.34% compilation time)

In [ ]:
0.117196 seconds (1.85 k allocations: 8.690 MiB)

## Hartree-Fock Calculation

In [ ]:
function getP(C, system)
    n, l = system.n, system.l
    
    P = la.zeros((l, l))
    for i in 1:n
        for a in 1:l
            for b in 1:l
                @inbounds P[b, a] += conj(C[a, i]) * C[b, i]
            end
        end
    end
    return P
end

function getF(P, system)
    h, u, n, l = system.h, system.u, system.n, system.l
    
    F = la.zeros((l, l))
    F .+= h
    for c in 1:l
        for d in 1:l
            @inbounds P_dc = P[d, c]
            for a in 1:l
                for b in 1:l
                    @inbounds F[a, b] += P_dc * u[a, c, b, d]
                end
            end
        end
    end
    return F
end

function SCF(C, system, iters)
    for i in 1:iters
        P = getP(C, system)
        F = getF(P, system)
        C = la.eigvecs(F)
    end
    
    return C
end

In [ ]:
#C_initial = ComplexF64.(la.I(system.l));
C_initial = la.Matrix(Float64.(la.I(system.l)))

@time C = SCF(C_initial, system, 20000);

In [ ]:
3.489390 seconds (360.00 k allocations: 945.435 MiB, 1.45% gc time)

## Results

In [ ]:
ψ = [abs.(i).^2 for i in (system.spfs .* C[:, 1])] .+ [abs.(i).^2 for i in (system.spfs .* C[:, 2])]
ψ = sum(ψ);

x_trans, y_trans = imgplot(file     = "Images/GS.PNG",
                           extent   = [-6, 6, 0, 0.4],
                           imgalpha = 0.5,
                           yticks   = [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4],
                           xticks   = [x for x in -6:2:6],
                           )

plt.plot!(x_trans.(grid), y_trans.(ψ))

In [ ]:
function expected_energy(C, system)
    l = system.l
    P = getP(C, system)
    h = system.h
    u = system.u
    
    energy = 0.0
    for a in 1:l
        for b in 1:l
            @inbounds energy += P[b, a] * h[a, b]
            for c in 1:l
                for d in 1:l
                    @inbounds energy += 0.5 * P[b, a] * P[d, c] * u[a, c, b, d]
                end
            end
        end
    end
    return real(energy)
end

In [ ]:
expected_energy(C, system)